In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:158: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:42: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:49: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [6]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


# Annual portfolio sorts

In [7]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,2)
sector_portfolio_sorts['All']=quantile_portfolios_annual(data,cleansed,2)

tsmom_model.py:238: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
tsmom_model.py:239: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [28]:
for sec in sector_portfolio_sorts.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())


Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.68 &   0.93 &   -1.10 \\
Standard Deviation           &  15.17 &  15.41 &   37.57 \\
Information Ratio            &   0.53 &   0.73 &   -0.35 \\
Skewness                     &  -0.08 &   0.45 &   -0.27 \\
Excess Kurtosis              &   1.27 &   4.22 &    1.00 \\
CAPM Alpha Annualized (in \%) &   4.46 &   7.51 &   -5.87 \\
CAPM Alpha Tstat             &   2.97 &   5.03 &   -1.02 \\
CAPM Beta (in \%)             &   1.01 &   1.01 &   -2.01 \\
CAPM Beta Tstat              &  25.46 &  19.00 &  -12.69 \\
\$R\textasciicircum2\$                        &   0.81 &   0.79 &    0.52 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.45 &   0.62 &   -0.27 \\
Standard Deviation           &   8.99 &   9.23 &   11.94 \\
Information Ratio            &   0.60 &   0.81 &   -0.27 \\
Skewness            

# FHT Annual portfolio sorts

In [30]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

In [31]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,2)
sector_portfolio_sorts_FHT['All']=quantile_portfolios_annual(data,cleansed,2)

In [32]:
for sec in sector_portfolio_sorts_FHT.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts_FHT[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.55 &   0.65 &    0.10 \\
Standard Deviation           &  13.46 &  14.60 &   12.74 \\
Information Ratio            &   0.49 &   0.53 &    0.09 \\
Skewness                     &  -0.14 &   0.25 &    0.15 \\
Excess Kurtosis              &   1.64 &   2.68 &    1.10 \\
CAPM Alpha Annualized (in \%) &   5.02 &   6.06 &    1.04 \\
CAPM Alpha Tstat             &   4.11 &   4.52 &    0.43 \\
CAPM Beta (in \%)             &   0.97 &   1.05 &    0.08 \\
CAPM Beta Tstat              &  34.89 &  22.75 &    1.18 \\
\$R\textasciicircum2\$                        &   0.78 &   0.77 &    0.00 \\
\bottomrule
\end{tabular}

All


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.37 &   0.56 &    0.19 \\
Standard Deviation           &   7.47 &   8.69 &    6.08 \\
Information Ratio            &   0.60 &   0.78 &    0.37 \\
Skewness                     &  -1.16 &  -0.74 &   -0.10 \\
Excess Kurtosis              &   6.26 &   3.69 &    0.23 \\
CAPM Alpha Annualized (in \%) &   2.48 &   4.39 &    1.91 \\
CAPM Alpha Tstat             &   3.94 &   6.53 &    1.61 \\
CAPM Beta (in \%)             &   0.87 &   1.04 &    0.16 \\
CAPM Beta Tstat              &  35.90 &  27.40 &    3.14 \\
\$R\textasciicircum2\$                        &   0.82 &   0.85 &    0.04 \\
\bottomrule
\end{tabular}

Currencies
\begin{tabular}{lrrr}
\toprule
{} &     0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        & -0.12 &   0.05 &    0.17 \\
Standard Deviation           &  6.20 &   7.20 &    8.24 \\
Information Ratio            & -0.23 &   0.09 &    0.24 \\
Skewness         

# Monthly portoflio sorts

In [38]:
amihud_monthly = read_monthly(amihud=True)

In [34]:
for sec in amihud_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(amihud_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(amihud_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.58 &   0.85 &    0.27 \\
Standard Deviation           &  14.79 &  16.38 &   12.95 \\
Information Ratio            &   0.47 &   0.62 &    0.25 \\
Skewness                     &  -0.06 &   0.37 &    0.12 \\
Excess Kurtosis              &   1.52 &   2.82 &    1.71 \\
CAPM Alpha Annualized (in \%) &   3.96 &   6.81 &    2.85 \\
CAPM Alpha Tstat             &   2.74 &   4.61 &    1.04 \\
CAPM Beta (in \%)             &   0.96 &   1.07 &    0.12 \\
CAPM Beta Tstat              &  24.73 &  19.16 &    1.36 \\
\$R\textasciicircum2\$                        &   0.80 &   0.82 &    0.01 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.45 &   0.62 &    0.17 \\
Standard Deviation           &   8.99 &   9.23 &    5.76 \\
Information Ratio            &   0.60 &   0.81 &    0.36 \\
Skewness            

# Monthly FHT

In [60]:
FHT_monthly = read_monthly(amihud=False)

In [61]:
for sec in FHT_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(FHT_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(FHT_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())



Agriculturals
\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.73 &   0.37 &   -0.37 \\
Standard Deviation           &  13.04 &  15.42 &   13.95 \\
Information Ratio            &   0.68 &   0.29 &   -0.32 \\
Skewness                     &   0.00 &   0.30 &    0.34 \\
Excess Kurtosis              &   1.16 &   3.91 &    2.38 \\
CAPM Alpha Annualized (in \%) &   7.98 &   2.65 &   -5.33 \\
CAPM Alpha Tstat             &   5.14 &   1.53 &   -1.68 \\
CAPM Beta (in \%)             &   0.90 &   1.11 &    0.21 \\
CAPM Beta Tstat              &  22.94 &  20.09 &    2.36 \\
\$R\textasciicircum2\$                        &   0.77 &   0.80 &    0.03 \\
\bottomrule
\end{tabular}

All


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


\begin{tabular}{lrrr}
\toprule
{} &      0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.50 &   0.48 &   -0.02 \\
Standard Deviation           &   7.69 &   8.84 &    6.77 \\
Information Ratio            &   0.78 &   0.65 &   -0.04 \\
Skewness                     &  -0.65 &  -0.91 &   -0.05 \\
Excess Kurtosis              &   4.44 &   4.54 &    0.77 \\
CAPM Alpha Annualized (in \%) &   4.04 &   3.28 &   -0.75 \\
CAPM Alpha Tstat             &   5.09 &   4.27 &   -0.51 \\
CAPM Beta (in \%)             &   0.89 &   1.04 &    0.15 \\
CAPM Beta Tstat              &  26.04 &  32.47 &    3.15 \\
\$R\textasciicircum2\$                        &   0.86 &   0.88 &    0.04 \\
\bottomrule
\end{tabular}

Currencies
\begin{tabular}{lrrr}
\toprule
{} &     0 &      1 &  Factor \\
\midrule
Monthly Return (in \%)        & -0.05 &   0.03 &    0.09 \\
Standard Deviation           &  6.15 &   6.83 &    7.85 \\
Information Ratio            & -0.10 &   0.05 &    0.13 \\
Skewness         

In [56]:
FHT_monthly = read_monthly(amihud=False)
sector='Currencies'
un_dec=FHT_monthly['Currencies']
sector_rtn=np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all')
    
    
un_dec['Factor']=un_dec[un_dec.columns[-1]]-un_dec[un_dec.columns[0]]
ind=un_dec['2000':.dropna(how='all').index
# AR(1) first
ex=un_dec.dropna(how='all')
en=ex.shift(-1).dropna()
en['Intercept']=1
ex=ex.ix[en.index]
r2=[]
coef=[]
tstat=[]
for i in un_dec.columns:
    res=sm.OLS(ex[str(i)],en[[str(i)]]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    coef.append(res.params[str(i)])
    tstat.append(res.tvalues[str(i)])        
ar1=pd.DataFrame()
ar1['Coef']=pd.Series(coef,index=un_dec.columns)
ar1['Tstats']=pd.Series(tstat,index=un_dec.columns)
# CAPM regression
capm_factor=pd.DataFrame()
if sector == 'All':
    capm_factor['Mkt-RF']=sector_rtn.mean(axis=1)
else:
    capm_factor['Mkt-RF']=sector_rtn[d_map[sector]].mean(axis=1)
capm_factor['Intercept']=1
alpha=[]
beta=[]
tstat_alpha=[]
tstat_beta=[]
for i in un_dec.columns:
    res=sm.OLS(un_dec.dropna()[str(i)].loc[ind],capm_factor[['Intercept','Mkt-RF']].loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    alpha.append(res.params['Intercept'])
    beta.append(res.params['Mkt-RF'])
    tstat_alpha.append(res.tvalues['Intercept'])
    tstat_beta.append(res.tvalues['Mkt-RF']) 
    r2.append(res.rsquared_adj)  
CAPM=pd.DataFrame()
CAPM['Alpha']=pd.Series(alpha,index=un_dec.columns)
CAPM['Alpha Tstat']=pd.Series(tstat_alpha,index=un_dec.columns)
CAPM['Beta']=pd.Series(beta,index=un_dec.columns)
CAPM['Beta Tstat']=pd.Series(tstat_beta,index=un_dec.columns)
CAPM['r2']=pd.Series(r2,index=un_dec.columns)



/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [57]:
print res.summary()

ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [58]:
un_dec.dropna()[str(i)].loc[ind]


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


1997-12-31         NaN
1998-01-31    0.038314
1998-02-28    0.008473
1998-03-31   -0.016367
1998-04-30    0.006246
1998-05-31    0.011128
1998-06-30   -0.017305
1998-07-31   -0.045198
1998-08-31    0.030761
1998-09-30    0.052006
1998-10-31    0.052764
1998-11-30    0.003139
1998-12-31    0.033744
1999-01-31   -0.008923
1999-02-28   -0.002857
1999-03-31   -0.015410
1999-04-30    0.018410
1999-05-31   -0.008949
1999-06-30    0.008545
1999-07-31   -0.019436
1999-08-31    0.021385
1999-09-30   -0.017848
1999-10-31   -0.003545
1999-11-30   -0.029969
1999-12-31    0.004645
2000-01-31   -0.015602
2000-02-29   -0.008838
2000-03-31    0.007715
2000-04-30    0.006473
2000-05-31    0.013280
                ...   
2014-08-31    0.006262
2014-09-30   -0.002891
2014-10-31   -0.001127
2014-11-30   -0.000026
2014-12-31   -0.001933
2015-01-31    0.072822
2015-02-28   -0.002545
2015-03-31   -0.019840
2015-04-30   -0.012974
2015-05-31   -0.027920
2015-06-30    0.020209
2015-07-31    0.029177
2015-08-31 

In [ ]:
capm_factor[['Intercept','Mkt-RF']].loc[ind]

In [59]:
def portfolio_sort_table(un_dec,sector_rtn,d_map,sector):
    un_dec['Factor']=un_dec[un_dec.columns[-1]]-un_dec[un_dec.columns[0]]
    ind=un_dec['2000':'2016'].dropna(how='all').index
    # AR(1) first
    ex=un_dec.dropna(how='all')
    en=ex.shift(-1).dropna()
    en['Intercept']=1
    ex=ex.ix[en.index]
    r2=[]
    coef=[]
    tstat=[]
    for i in un_dec.columns:
        res=sm.OLS(ex[str(i)],en[[str(i)]]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        coef.append(res.params[str(i)])
        tstat.append(res.tvalues[str(i)])        
    ar1=pd.DataFrame()
    ar1['Coef']=pd.Series(coef,index=un_dec.columns)
    ar1['Tstats']=pd.Series(tstat,index=un_dec.columns)
    # CAPM regression
    capm_factor=pd.DataFrame()
    if sector == 'All':
        capm_factor['Mkt-RF']=sector_rtn.mean(axis=1)
    else:
        capm_factor['Mkt-RF']=sector_rtn[d_map[sector]].mean(axis=1)
    capm_factor['Intercept']=1
    alpha=[]
    beta=[]
    tstat_alpha=[]
    tstat_beta=[]
    for i in un_dec.columns:
        res=sm.OLS(un_dec.dropna()[str(i)].loc[ind],capm_factor[['Intercept','Mkt-RF']].loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        alpha.append(res.params['Intercept'])
        beta.append(res.params['Mkt-RF'])
        tstat_alpha.append(res.tvalues['Intercept'])
        tstat_beta.append(res.tvalues['Mkt-RF']) 
        r2.append(res.rsquared_adj)  
    CAPM=pd.DataFrame()
    CAPM['Alpha']=pd.Series(alpha,index=un_dec.columns)
    CAPM['Alpha Tstat']=pd.Series(tstat_alpha,index=un_dec.columns)
    CAPM['Beta']=pd.Series(beta,index=un_dec.columns)
    CAPM['Beta Tstat']=pd.Series(tstat_beta,index=un_dec.columns)
    CAPM['r2']=pd.Series(r2,index=un_dec.columns)
    # Presentation
    res=pd.DataFrame()
    res['Monthly Return (in %)']=un_dec.mean()*100
    res['Standard Deviation']=un_dec.std()*math.sqrt(12)*100
    res['Information Ratio']=calc_Sharpe(un_dec)
    res['Skewness']=un_dec.skew()
    res['Excess Kurtosis']=un_dec.kurtosis()
    #res['AR(1)']=ar1.Coef
    #res['AR(1) Tstat']=ar1.Tstats
    res['CAPM Alpha Annualized (in %)']=CAPM.Alpha*1200
    res['CAPM Alpha Tstat']=CAPM['Alpha Tstat']
    res['CAPM Beta (in %)']=CAPM.Beta
    res['CAPM Beta Tstat']=CAPM['Beta Tstat']
    res['$R^2$']=CAPM.r2.abs()
    res =res.round(2)
    return res.T